# Lecture 7

In [1]:
reply="ciao"
k=0
while reply!="Stop" and k<3:
    try:
        print(int(reply)**2)
    except:
        print("Bad"*8)
    reply="new impute"
    k+=1
print("bye")


BadBadBadBadBadBadBadBad
BadBadBadBadBadBadBadBad
BadBadBadBadBadBadBadBad
bye


In [2]:
try:
    raise ZeroDivisionError
except:
    print("ciao")

ciao


In [7]:
class MyExcp(Exception):
    pass
try:
    raise MyExcp(100)
except MyExcp as x:
    #is local
    print("caught",x)
#print(x,"again")#will raise an error    

caught 100


In [9]:
class MyExcp(Exception):
    pass
P=(MyExcp,IndexError)
try:
    raise P[0]
except MyExcp:
    print("caught") 

caught


In [10]:
class MyExcp(Exception):pass
class MySpec1(MyExcp):pass
class MySpec2(MyExcp):pass
for e in (MyExcp(0),MySpec1(1),MySpec2(2)):
    try:
        raise e
    except MyExcp as x:
        print("caught")

caught
caught
caught


In [14]:
class B(Exception): pass
class C(B): pass
class D(B):pass
for i in [B,C,D]:
    try:
        raise i()
    except B:
        print("D")
    except C:
        print("C")
    except B:
        print("B")

D
D
D


# How much an operation cost in python

The time need to perform an operation is indipendet from the value of the operation 

There is no max value of maximum integer

In [21]:
#do not insert prints or tests
#implement the methods instead of "pass"

from datetime import datetime

class Account:
    def __init__(self, bank, number, holder, opening_balance=0.0):
        self.bank = bank
        self.number = number
        self.holder = holder
        self.__balance = float(opening_balance)
    def set_balance(self,balance):
        self.__balance=float(balance)
        return None
    
    def get_balance(self):
        return self.__balance
    
    def set_balance(self, balance):
        self.__balance=float(balance)

    def deposit(self, amount):
        if amount <= 0:
            return 0
        self.__balance += float(amount)
        return amount

    def withdraw(self, amount):
        if amount <= 0:
            return 0
        if amount <= self.__balance:
            self.__balance -= amount
            return amount
        else:
            everything = self.__balance
            self.__balance = 0
            print("Insufficient funds")
            return everything
    
    def __lt__(self, other):
        #for sorting purposes
        return self.number < other.number



class Holder:
    def __init__(self, ident, name, surname):
        self.ident = ident
        self.name = name
        self.surname = surname
        self.__accounts = {}

    def add_account(self, account):
        if (account.bank, account.number) not in self.__accounts:
            self.__accounts[account.bank, account.number] = account

    def total_balance(self):
        return sum(c.get_balance() for c in self.__accounts.values())
        
    def __lt__(self, other):
        #for sorting purposes
        return self.surname < other.surname
        

class Bank:
    def __init__(self, name):
        self.name = name
        self.__last_created_account = 0
        self.__holders = {}
        self.__accounts = {}

    def print_holders(self):
        print("Holders of bank", self.name)
        for c in sorted(self.__holders.values()):
            print(c.ident, c.name, c.surname)

    def print_accounts(self):
        print("Accounts of bank", self.name)
        for c in sorted(self.__accounts.values()):
            print(c.number, c.holder.ident, c.get_balance())

    def new_account(self, holder, initial_balance=0, tipology="standard"):
        if isinstance(holder, Holder) and initial_balance >= 0:
            self.__last_created_account += 1
            if tipology == 'interests':
                nc = Account_with_interests(self, self.__last_created_account, holder, initial_balance)
            elif tipology == 'overdraft':
                nc = Account_with_overdraft(self, self.__last_created_account, holder, initial_balance)
            else:
                nc = Account(self, self.__last_created_account, holder, initial_balance)
            self.__accounts[nc.number] = nc
            if holder.ident not in self.__holders:
                self.__holders[holder.ident] = holder   
            holder.add_account(nc)
            return nc
    
    def __get_account(self, account_number):
        return self.__accounts.get(account_number, None)

    def deposit(self, account_number, amount):
        c = self.__get_account(account_number)
        if c is None:
            print("Account not available")
        else:
            c.deposit(amount)
            print("Deposited", c, "on account", account_number)

    def withdraw(self, account_number, amount):
        c = self.__get_account(account_number)
        if c is None:
            print("Account not available")
            return
        p = c.withdraw(amount)
        print("Withdrawn", p, "from account", account_number)
    
    def same_bank_transfer(self, debit_account, credit_account, amount):
        da = self.__get_account(debit_account)
        ca = self.__get_account(credit_account)
        if da is None:
            print("Debit account not available")
            return
        if ca is None:
            print("Credit account not available")
            return
        if da.get_balance() < amount:
            print("Insufficient funds")
            return
        da.withdraw(amount)
        ca.deposit(amount)
    
    def another_bank_transfer(self, debit_account, credit_bank, credit_account, amount):
        da = self.__get_account(debit_account)
        if da is None:
            print("Debit account not available")
            return
        if da.get_balance() < amount:
            print("Insufficient funds")
            return
        if not isinstance(credit_bank, Bank):
            print("Credit bank not available")
            return
        da.withdraw(amount)
        credit_bank.deposit(credit_account, amount)




class Account_with_interests(Account):
    import datetime
    def __init__(self, bank, number, holder, opening_balance=0, rate=0.03, n=4):
        super().__init__(bank, number, holder, opening_balance)
        self.rate=rate
        self.n=n
        self.__last_year=datetime.now().year
        
    def add_interests(self, year):
        if year>self.__last_year:
            d=year-self.__last_year
            balance=self.get_balance()*(1+(self.rate/self.n))**(self.n*d)
            self.set_balance(balance)
        

class Account_with_overdraft(Account):
    def __init__(self, bank, number, holder, opening_balance=0, overdraft=1000):
        super().__init__(bank, number, holder, opening_balance)
        self.__overdraft=overdraft
        
    def withdraw(self, amount):
        if amount <= 0:
                return 0
        everything = self.get_balance() + self.__overdraft
        if amount <= everything:
            self.set_balance(self.get_balance() - amount)
            return amount
        else:
            self.set_balance(-self.__overdraft)
            print("Insufficient funds")
            return everything



#DO NOT EDIT FROM THIS LINE BELOW

ms = Holder('mrcsbr', 'Marco', 'Sbaraglia')
ban_bo = Bank("Bank Bologna")
ban_rav = Bank("Bank Ravenna")
cnt1 = ban_bo.new_account(ms, 1000, 'interests')
cnt2 = ban_rav.new_account(ms, 10000, 'interests')
cnt3 = ban_bo.new_account(ms, 100)
print(ms.total_balance())
cnt1.add_interests(2020)
print(cnt1.get_balance())
cnt1.add_interests(2021)
print(cnt1.get_balance())
cnt1.add_interests(2022)
print(cnt1.get_balance())
cnt2.add_interests(2022)
print(cnt2.get_balance())
cnt1.add_interests(2024)
print(cnt1.get_balance())
print(ms.total_balance())


cnt11 = ban_bo.new_account(ms, 1000, 'overdraft')
cnt12 = ban_rav.new_account(ms, 500, 'overdraft')
cnt13 = ban_bo.new_account(ms, 100)
print(ms.total_balance())
print(cnt11.get_balance())
cnt11.withdraw(100)
print(cnt11.get_balance())
cnt11.withdraw(1000)
print(cnt11.get_balance())
cnt11.withdraw(-200) #non ha effetto
print(cnt11.get_balance())
cnt11.withdraw(1100)
print(cnt11.get_balance())
cnt11.withdraw(1100)
print(cnt11.get_balance())
ban_bo.withdraw(cnt11.number,-100) #non ha effetto
print(cnt11.get_balance())
ban_bo.withdraw(cnt11.number,2000)
print(cnt11.get_balance())
print(cnt13.get_balance())
cnt13.withdraw(100)
print(cnt13.get_balance())
cnt13.withdraw(50)

11100.0
1000.0
1000.0
1000.0
10000.0
1061.5988478182758
11161.598847818275
12761.598847818275
1000.0
900.0
-100.0
-100.0
Insufficient funds
-1000.0
Insufficient funds
-1000.0
Withdrawn 0 from account 3
-1000.0
Insufficient funds
Withdrawn 0.0 from account 3
-1000.0
100.0
0.0
Insufficient funds


0.0

In [27]:
class New_Fraction:
    #INSERT INIT HERE
    def __init__(self,N,D=1):
        if type(N)!= int or type(D)!= int:
                raise TypeError("non integer numerator or denominator")
        elif D==0:
                raise ZeroDivisionError("denominator is 0")
        elif D<0:
                raise ValueError("negative denominator, sign should be in numerator")
        else:
            self.__num=N
            self.__den=D

    #DO NOT MODIFY OR DELETE __str__ METHOD
    def __str__(self):
        return str(self.__num) + "/" + str(self.__den)

#TEST: DO NOT MODIFY CODE BELOW
L = [(1,4),(-3,'c'),(-3,0),(-3,-2),(-3,2),(5,1),(8,9)]
F = []
for (num, den) in L:
    try:
        f = New_Fraction(num,den)
        F.append(f)
    except Exception as e:
        print(str(num), str(den), "Fraction NOT created:", type(e).__name__, "-", e)

print("Created fractions are: ")
for f in F:
    print(f)

-3 c Fraction NOT created: TypeError - non integer numerator or denominator
-3 0 Fraction NOT created: ZeroDivisionError - denominator is 0
-3 -2 Fraction NOT created: ValueError - negative denominator, sign should be in numerator
Created fractions are: 
1/4
-3/2
5/1
8/9


In [ ]:
class NegativeIntegerError(Exception): pass
class Terminate(Exception): pass

tot = 0
n = 0
try:
    while True:  
        try:
            r = int(input("R:\n"))
            if r == 99999:
                raise Terminate
            if r < 0:
                raise NegativeIntegerError
            tot += r
            n += 1   
        except ValueError:
            print("II") #invalid integer
        except NegativeIntegerError:
            print("NI") #negative integer
except Terminate:
    try:
        print(round(tot/n, 2))
    except ZeroDivisionError:
        print("ZERO")


In [16]:
import numpy as np
def merge(a,b):
    b=b.reshape(a.shape)
    b[b<a]=0
    a[a<b]=0
    return a+b
print(merge(np.array([[0,20,4],[7,1,3]]),np.array([0,1,2,3,4,5])))

[[ 0 20  4]
 [ 7  4  5]]


In [19]:
import numpy as np

def demean(a):
    b=np.mean(a,axis=0)
    c=a-b
    return c
    
print(demean(np.array([[7,2,4],[5,1,8]])))

[[ 1.   0.5 -2. ]
 [-1.  -0.5  2. ]]


In [25]:
import numpy as np
def colsum(a):
    
    b=np.sum(np.sum(a[:,::2],axis=0))
    c=np.sum(np.sum(a[:,1::2],axis=0))
    return np.array((b,c))

print(colsum(np.array([[2, 7, 8, 0],
                       [3, 9, 3, 4],
                       [4, 0, 1, 3]])))

[21 23]


In [27]:
import numpy as np

def count_common(a,b):
    #begin student code
    return len(a[a==b])


print(count_common(np.array([[1,3,8],
              [4,5,2],
              [3,2,9]]),np.array([[2,3,7],
              [4,5,9],
              [6,6,6]])))

3


In [30]:
import numpy as np
def checkerboard(m):
    def f(i,j):
        if i%2==0:
            if (i+j)%2==0:
                return 0
            else:
                return 1
        else:
            if (i+j)%2==1:
                return 1
            else:
                return 0
    f=np.vectorize(f)        
    a=np.fromfunction(f,(m,m))
    return a
print(checkerboard(10))

[[0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]]


In [55]:
import numpy as np
def zerocolumns(a):
    b=np.prod(a,axis=0)!=0
    return a[:,b]
    
    
print(zerocolumns(np.array([[1,2,0],[3,1,1],[3,3,0]])))       

[[1 2]
 [3 1]
 [3 3]]


In [56]:
import numpy as np
def arraydistance(p,q):
    def f(i,j):
        return np.sqrt(i**2+j**2)
    return np.fromfunction(f,(p,q))
print(arraydistance(3,3))


[[0.         1.         2.        ]
 [1.         1.41421356 2.23606798]
 [2.         2.23606798 2.82842712]]


In [100]:
#do not insert print or tests
import numpy as np
def exchangecolumns(a):
    if len(a[0,:])%2!=0:
        return a
    b=np.copy(a)
    a[:,0::2],a[:,1::2]= b[:,1::2],b[:,0::2]
    return a
   



print(exchangecolumns(np.arange(0,20).reshape(5,4)))

[[ 1  0  3  2]
 [ 5  4  7  6]
 [ 9  8 11 10]
 [13 12 15 14]
 [17 16 19 18]]


In [5]:
import numpy as np
def nonzero(m):
    return m[:,np.prod(m,axis=0)!=0]
print(nonzero(np.array([[1,2,3],[3,4,0]])))

[[1 2]
 [3 4]]


In [11]:
import numpy as np
def modify(a, n):
    #insert only one line here
    a[a>n]=a[a>n]+np.sum(a[a<n])
    #end student code
    return a


a = np.array([[4, 5, 6], [7,8,9], [10, 11, 12], [17, 18, 19]])
print(modify(a, 8))

[[ 4  5  6]
 [ 7  8 31]
 [32 33 34]
 [39 40 41]]


In [39]:
L=[1,2,3,-4]
i=0
while i<len(L):
        if L[i] <0 :
            del L[i]
        elif L[i]>=0:
            L[i+1:i+1]=[0]*L[i]
            i+=1
        print(L,"******",i)   
print(L)

[1, 0, 2, 3, -4] ****** 1
[1, 0, 2, 3, -4] ****** 2
[1, 0, 2, 0, 0, 3, -4] ****** 3
[1, 0, 2, 0, 0, 3, -4] ****** 4
[1, 0, 2, 0, 0, 3, -4] ****** 5
[1, 0, 2, 0, 0, 3, 0, 0, 0, -4] ****** 6
[1, 0, 2, 0, 0, 3, 0, 0, 0, -4] ****** 7
[1, 0, 2, 0, 0, 3, 0, 0, 0, -4] ****** 8
[1, 0, 2, 0, 0, 3, 0, 0, 0, -4] ****** 9
[1, 0, 2, 0, 0, 3, 0, 0, 0] ****** 9
[1, 0, 2, 0, 0, 3, 0, 0, 0]


In [41]:
class Top:
    def __init__(self,c): #DO NOT modify __init__
        if type(c)!=int:
            raise TypeError
        self.__n=c
    def get(self):
            return self.__n
    #complete here the def of Top, as needed
class A(Top):
    def __eq__(self,other):
        return self.get()==other.get()
    #complete here the def of A, as needed
class B(A):
    def __eq__(self,other):
        return abs(self.get())==abs(self.get())
    #complete here the def of B, as needed

# DO NOT MODIFY THE FOLLOWING
a1=A(1)
a2=A(-1)

print(a1==a2)

b1=B(1)
b2=B(-1)

print(b1==b2)

False
True


In [43]:
#do not insert print or tests
import numpy as np

def select(a, b, n):
    #BEGIN STUDENT CODE HERE
    if a.size != b.size:
        return a
    b=b.reshape(a.shape)
    c= a[b<n]
    return np.ravel(c)
    #END STUDEND CODE HERE
    
#DO NOT EDIT BELOW

#test 1
a = np.array([[0, 4, 3, 7],
              [3, 2, 4, 4],
              [6, 6, 6, 2]])
b = np.array([1, 2, 9, 5, 2, 9, 6, 7, 7, 2, 3, 2])

print(select(a, b, 6))

#test 2

a = np.array([[3, 7],
              [1, 3]])

b = np.array([[5, 2],
              [0, 9]])

print(select(a, b, 5))

#test 3
a = np.array([7, 6, 1, 3, 5, 2, 4, 2, 0])

b = np.array([[6, 2],
              [3, 0],
              [3, 5],
              [9, 9],
              [8, 1]])

print(select(a, b, 5))

[0 4 7 3 6 6 2]
[7 1]
[7 6 1 3 5 2 4 2 0]


In [45]:
class A:
    def __init__(self, xx): 
        self.__x = xx
    def get(self): 
        return self.__x
class B(A):
    pass

class C(B):
    def __init__(self,x):
        super().__init__(x)
    def get(self):
        return super().get()
        
    def __add__(self,other):
        return C(self.get()+other.get())
    def __str__(self):
        return str(self.get())
    #INSERT YOUR OTHER METHOD DEFINITIONS, if needed


#DO NOT MODIFY FROM HERE!
b=C(10)
c=C(20)
print(b+c)
b=C(70)
c=C(7)
print(b+c)

30
77


In [52]:
def fexp(L):
    if len(L)<=2:
        return True
    sum=L[0]
    var=True
    for i in range(1,len(L)):
        if L[i]==sum:
            sum+=L[i]
        else:
            var=False
    if var:
        return True
    else:
        sum=L[0]
        for i in range(len(L)):
            if L[i]==sum:
                sum+=L[i]-sum
            else:
                L[i]=sum
                sum+=sum
        return False
exec('LL=[1,2,3] \nprint(fexp(LL)) \nprint(LL)')
exec('LL=[1,1,2,4,8,16] \nprint(fexp(LL)) \nprint(LL)')
                

False
[1, 1, 2]
True
[1, 1, 2, 4, 8, 16]


In [60]:
class SeqMod:
    def __init__(self,L):
        self.__LL=L
    def __str__(self):
        return str(self.__LL)
    def get(self):
        return self.__LL

    
class SubCl(SeqMod):
    def __init__(self,first,n):
        sum=first
        LL=[first]
        for i in range(1,n):
            LL.append(sum)
            sum+=sum
        super().__init__(LL)
        
        
print(SubCl(3,6))

[3, 3, 6, 12, 24, 48]


In [2]:
#do not insert print, or tests
def fine(S):
    sum_=0
    for i,e in enumerate(S):
        if e>0:
            if e<=sum_:
                return False
        sum_+=i
    return True

print(fine([-1,2,2,4,7]))

True


In [7]:
import numpy as np
'''
A parity matrix is a 2D NumPy array where the sum of each row is even and the sum of each column is even
Write a function erroneous(x) that,
given a parity matrix where a single number has been changed,
so that the matrix does not satisfy anymore the property of parity (in particular,
there is one row and one column whose sums are odd),
spots the erroneous item and returns a NumPy array of shape (1,) containing that erroneous number'''
def erroneus(x):
    b=np.sum(x,axis=0)%2!=0
    a=np.sum(x,axis=1)%2!=0
    return x[a,b]
   
a = np.array([
             [ 2,  4,  7,  9],
             [ 1,  5, 34, 17],
             [32, 34, 44, 54],
             [14, 15, 16, 19],
             [23, 24, 26, 25]])
print(erroneus(a))

[34]


In [9]:
class A:
    def __init__(self, xx): 
        self.__x = xx
    def get(self): 
        return self.__x
class B(A):
    pass

class C(B):
    def get(self):
        #INSERT HERE THE BODY OF get
        return super().get()
    def __add__(self,other):
        return C(self.get()+other.get())
    def __str__(self):
        return "C: "+ str(self.get())
print(C(10)+C(20))

C: 30


In [14]:
import numpy as np
def nonzero(m):
    return m[np.sum(m,axis=1)!=0,:]
a= np . array ([[1 , 2, 0] ,
[ -1 ,0 , 1] ,
[4 , 5, 6] ,
[7 , 8, 9]])
print(nonzero(a))

[[1 2 0]
 [4 5 6]
 [7 8 9]]


In [15]:
def permu(elements):
    if len(elements) <= 1:
        yield elements
        return 
    for perm in permu(elements[1:]):
        for i in range(len(elements)):
            # nb elements[0:1] works in both string and list contexts
            yield perm[:i] + elements[0:1] + perm[i:]
list(permu([1,2,3]))



[[1, 2, 3], [2, 1, 3], [2, 3, 1], [1, 3, 2], [3, 1, 2], [3, 2, 1]]

In [38]:
class S:
    def __init__(self,x):
        self.x=x
l=[]
for i in range(1,10,1):
    
    l.append(S(i))
l.append(S(0))
newlist=[]
for i in range(len((l))):
    for j in range(len(l)):
        if l[i].x<l[j].x:
            l[i],l[j]=l[j],l[i]        
print([i.x for i in l])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [3]:
def sum_pairs(L):
    L_1=[i[1] for i in L]
    repetition={}
    for e in L_1:
        if e not in repetition:
            repetition[e]=1
        else:
            repetition[e]=repetition.get(e,None)+1
    sum=0
    for k,v in repetition.items():
        if v!=1:
            for i in L:
                if i[1]==k:
                    sum+=i[0]            
    return sum
print(sum_pairs([ (1,2), (2,3), (3,2), (10,1), (-2, 2), (-1,3), (1,9) ]))       
    

3


In [17]:
class A:
    def __init__ ( self , x ):
        self.__myx =x
    def setx ( self ,x ):
        self . __myx =x
    def getx ( self ):
        return self . __myx
class B( A ):
    def __init__( self ,x ,y ):
        super().__init__ (x)
        self.__myy =y
    def f(self):
        return self.__myy
     
# MAY BE COMPLETED ; pass IF NOT COMPLETED
def morethanonce_y(LB):
    Ly=[i.f() for i in LB]
    L=[(i.getx(),i.f()) for i in LB]
    repetition={}
    for e in Ly:
        if e not in repetition:
            repetition[e]=1
        else:
            repetition[e]=repetition.get(e,None)+1
    L0=[]
    for k,v in repetition.items():
        if v!=1:
            for i in L:
                if i[1]==k:
                     L0.append((i[0],i[1]))         
    return L0
        
     
print(morethanonce_y([B(1,2), B(2,3), B(3,2), B(10,1), B(-2, 2), B(-1,3), B(1,9)]))

[(1, 2), (3, 2), (-2, 2), (2, 3), (-1, 3)]


In [8]:
import numpy as np
def nonzero(n,b):
    b=b[b!=0]  
    try:
        return b[:n**2].reshape(n,n)
    except:
         b=np.concatenate((b,np.zeros((n**2-len(b),))),axis=0)
         return b.reshape((n,n))
    
print(nonzero(3,np.array([[1 , 2 , 0],[4 , 5, 6]])))

[[1. 2. 4.]
 [5. 6. 0.]
 [0. 0. 0.]]


In [19]:
def smooth(S,lun):
    lista=[]
    for i in range(0,len(S)-lun+1):
        v=[S[j] for j in range(i,i+lun)]
        var=sum(v)/len(v)
        lista.append(round(var,2))
    return lista
print(smooth([1,2,10,12,4,8,6,2,10], 2))

[1.5, 6.0, 11.0, 8.0, 6.0, 7.0, 4.0, 6.0]


In [21]:
class MyList :
    def __init__ ( self , L ):
        self . __myL =L
        def myapp ( self ,x ):
            self . __myL . append (x)
    def getL ( self ):
        return self . __myL
class SC(MyList):
    def __init__(self,L):
        super().__init__(L)
    def average(self):
        if len(self.getL())==0:
            return None
        else:
            lenz=len(self.getL())
            return sum(self.getL())/lenz
print(SC([1,2,3,4,5]).average())

3.0


In [22]:
def nonzero(b):
  return b[np.prod(b,axis=1)!=0,:]



print(nonzero(np.array([[1 , 2 , 0] ,[4 , 5, 6] ,[7 , 8, 9]])))

[[4 5 6]
 [7 8 9]]


In [29]:
def mode(L):
    dic={}
    for i in L:
        if i not in dic:
            dic[i]=1
        else:
            dic[i]=dic.get(i)+1
    L=[]
    max=0
    for k,v in dic.items():
        if v>max:
            L=[]
            L.append(k)
            max=v
        elif v==max:
            L.append(k)
    return L
    
print(mode([1,1,2,3,2,2,2,3,4,5,5,5,5]))

[2, 5]


In [31]:
def nonzero(b):
    return b[np.prod(b,axis=1)>10,:]

print(nonzero(np.array([[1 , 2 , 0] ,[4 , 5, 6] ,[7 , 8, 9]])))


[[4 5 6]
 [7 8 9]]


In [34]:
class A:
    def __init__ ( self , x ):
        self . xx =x
def __str__ ( self ):
    return ( str ( self . xx ))
def mode(L):
    L=[i.xx for i in L]
    dic={}
    for i in L:
        if i not in dic:
            dic[i]=1
        else:
            dic[i]=dic.get(i)+1
    L=[]
    max=0
    for k,v in dic.items():
        if v>max:
            L=[]
            L.append(A(k))
            max=v
        elif v==max:
            L.append(A(k))
    return L
def imode(L1,L2):
    L1=mode(L1)
    L2=mode(L2)
    L1=[i.xx for i in L1]
    L2=[i.xx for i in L2]
    Lcommon=[]
    for i in L1:
        for j in L2:
            if i==j:
                Lcommon.append(A(i))
    return Lcommon
print(imode([A(1),A(4),A(5),A(4),A(6),A(5),A(4),A(1),A(4)],[A(1),A(4),A(5),A(6),A(5),A(4),A(1)]))

In [11]:
def palindrome(s):
    s=s.lower()
    sremove=[" ",".",",","!"]
    for i in sremove:
        s=s.replace(i,"")
    return s==s[::-1]
        

print(palindrome(',AM,......A'))

True


In [23]:
def compare(s1,s2):
    index=[]
    for i,e in enumerate(s1):
        if i==0 and (e==s2[i] or e==s2[i+1]):
            index.append(i)
        elif i==len(s2)-1 and (e==s2[i] or e==s2[i-1]):
            index.append(i)
        elif  i <len(s2)-1 and (e==s2[i] or e==s2[i-1] or e==s2[i+1]):
            index.append(i)
        elif i==len(s2)  and ( e==s2[i-1]):
            index.append(i)
    return index
        
        
    return index
print(compare('lasca', 'asca'))

[1, 2, 3, 4]


In [31]:
def floyd(k):
    number=1
    lista=[]
    for i in range(1,k+1):
        for j in range(1,i+1):
            lista.append(number)
            number+=1
        print(tuple(lista))
        lista=[]
floyd(5)

(1,)
(2, 3)
(4, 5, 6)
(7, 8, 9, 10)
(11, 12, 13, 14, 15)


In [54]:
#do not insert print or tests
def leap(year):
    if year%400==0:
        return 29
    return 28
def numberofdays(month,year):
    l30=[11,4,6,9]
    l28=leap(year)
    if month in l30:
        return 30
    elif month==2:
        return l28
    else:
        return 31
    
def magic(star,end):
    star=list(str(star))
    end=list(str(end))
    lista=[]
    star1=int(star[2]+star[3])
    end1=int(end[2]+end[3])
    if end1==0:
        end1=100
    print(star1,end1)
    for y in range(star1,end1+1):
         for m in range(1,13):
             for d in range(1,numberofdays(m,y)+1):
                 if d*m==y and y!=100:
                      lista.append((d,m,int(str(star[0]+star[1])+str(y))))
                 elif d*m==00 and y==100:
                     lista.append((d,m,int(str(star[0]+star[1])+str("00"))))
    return tuple(lista)
print(magic(1990,2000))
print(magic(1990,2000)==((30, 3, 1990), (18, 5, 1990), (15, 6, 1990), (10, 9, 1990), (9, 10, 1990), (13, 7, 1991), (23, 4, 1992), 
                         (31, 3, 1993), (19, 5, 1995), (24, 4, 1996), (16, 6, 1996), (12, 8, 1996),
                         (8, 12, 1996), (14, 7, 1998), (11, 9, 1999), (9, 11, 1999)))

90 100
((30, 3, 1990), (18, 5, 1990), (15, 6, 1990), (10, 9, 1990), (9, 10, 1990), (13, 7, 1991), (23, 4, 1992), (31, 3, 1993), (19, 5, 1995), (24, 4, 1996), (16, 6, 1996), (12, 8, 1996), (8, 12, 1996), (14, 7, 1998), (11, 9, 1999), (9, 11, 1999))
90 100
True


In [55]:
def collatz(n):
    if n==0:
        return []
    lista=[n]
    while n!=1:
        if n%2==0:
            n=n//2
        else:
            n=n*3+1
        lista.append(n)
    return lista 
	
print(collatz(19))

[19, 58, 29, 88, 44, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1]


In [69]:
def contiguous(L):
    newlist=[]
    count=1
    i=0
    while i<len(L):
       for j in range(i,len(L)):
           if i!=j:
               if L[i]==L[j]:
                   count+=1
               else:
                   break
       if count!=1:
           newlist.append(L[i])
           newlist.append(count)
       else:
           newlist.append(L[i])
       i+=count
       count=1
    return newlist
        
            
    
print(contiguous(['a','b','b','z','o','o','b','b','b','b','k']))

['a', 'b', 2, 'z', 'o', 2, 'b', 4, 'k']


In [74]:
def contiguous_inplace(L):
    if len(L)==0:
        return L
    newlist=[]
    count=1
    i=0
    while i<len(L):
       for j in range(i,len(L)):
           if i!=j:
               if L[i]==L[j]:
                   count+=1
               else:
                   break
       if count!=1:
           L[i:i+count]=[L[i],count]
       i+=count
       count=1
    return L

print(contiguous_inplace(['a','b','b','z','o','o','b','b','b','b','k'])) 

['a', 'b', 2, 'z', 'o', 2, 'b', 4, 'k']


In [ ]:
sum=0
n=0
x=0
while x!=9999:
    x=int(input("R:"))
    if x!=9999 and x>0:
        sum+=x
        n+=1
    
print(round(sum/n,2))

In [144]:
def repeat_inplace(L):
    i=0
    while i<len(L):
        a=L[i]
        L[i:i+1]=L[i]*[L[i]]
        i+=a
    return L
A=[3,2,0,4]
print(repeat_inplace(A))


[3, 3, 3, 2, 2, 4, 4, 4, 4]


In [154]:
#do not use print or test
def dict_to_str(D):
    newd={}
    for k,v in D.items():
        for j in v:
            newd[j]=k
    stringa=""
    l=list(newd.values())
    for k,v in newd.items():
        l[k]=v
    for i in l:
        stringa+=i
    return stringa
	
print(dict_to_str({'m': [0,2], 'o': [1], '!':[3]}))

mom!


In [160]:
def merge(LD): #LD list of dictionaries
    for i in LD:
         if type(i)!= dict:
             return None
    newd={}
    for d in LD:
        for k,v in d.items():
            if k not in newd:
                newd[k]=[v]
            else:
                if v not in newd.get(k):
                    newd[k]=newd.get(k)+[v]
    n={}
    for k,v in newd.items():
        if len(v)==1:
            for i in v:
                newd[k]=i
    return newd
print(merge([{(1,2):5,(4,5):8,(1,2):6}, {(1,3):0,(4,5):8,(1,2):-1}]) )

{(1, 2): [6, -1], (4, 5): 8, (1, 3): 0}


In [162]:
def invert(D):
    return {v:[k2 for k2 in D.keys() if D.get(k2)==v] for k,v in D.items()}
print(invert({'a':2, 'b':2, 'c':3, 'd':4, 'e':3,'f':0, 'g':3, 'h':2}))

{2: ['a', 'b', 'h'], 3: ['c', 'e', 'g'], 4: ['d'], 0: ['f']}


In [174]:
import numpy as np
def merge(a,b):
    b=b.reshape(a.shape)
    return np.concatenate((a[a>=b],b[b>a]),axis=0).reshape(a.shape)
print(merge(np.array([[0,20,4],[7,1,3]]),np.array([0,1,2,3,4,5])))

[[ 0 20  4]
 [ 7  4  5]]


In [177]:
import numpy as np

def demean(a):
    media=np.mean(a,axis=0)
    b=a-media
    return b
print(demean(np.array([[1,2,3],
                [4,5,6],
                [7,8,9],
                [10,11,12]])))

[[-4.5 -4.5 -4.5]
 [-1.5 -1.5 -1.5]
 [ 1.5  1.5  1.5]
 [ 4.5  4.5  4.5]]


In [184]:
import numpy as np

def count_common(a,b):
    return len(a[a==b])

print(count_common(np.array([[1,3,8],[4,5,2],[3,2,9]]),
                   np.array([[2,3,7],[4,5,9],[6,6,6]])))


[3 4 5]


In [195]:
import numpy as np
def zeroones(p,q):
     a=np.arange(1,(p-2)*(q-2)+1,).reshape((p-2,q-2))
     c=np.zeros((p,q))
     c[1:p-1,1:q-1]=a
     return c
print(zeroones(4,8))

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  2.  3.  4.  5.  6.  0.]
 [ 0.  7.  8.  9. 10. 11. 12.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


In [200]:
L=[1,2,3,4,5,6,7,8]
L[8:10]=[9]
print(L)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [207]:
class Try(Exception):pass
class Drip(Try):pass
try:
    raise Try("hola")
except Drip as x:
    print(x)
except Try as x:
    print("bella")
except Drip as x:
    print(x)

bella


In [211]:
from abc import ABC
class C(ABC):
    def __init__(self,other):
        pass
    def s(self):
        return 1
print(C(1))

In [215]:
import numpy as np
x=np.array([1,2,3,4,5])
print(x.max())
print(x.prod())
print(x[x<20])

5
120
[1 2 3 4 5]
